### CONTENTS
#### 1 Importing Libraries
#### 2 Importing Data 
#### 3 Data wDangling
#### 4 Reshape Data for Deep Learning
#### 5 Keras Model CNN
##### 5.1 Model 1
###### 5.1.1 Model 1 Confusion Matrix
##### 5.2 Model 2
##### 5.3 Model 3
##### 5.4 Model 4
##### 5.5 Model 5
##### 5.6 Model 6: Extra Convolution Layer
##### 5.7 Model 7: Extra Convolution Layer and Dropout
##### 5.8 Model 8: Extra Pooling Layer
##### 5.9 Model 9: More Filters
##### 5.10 Model 10: More Epochs

# 1 Importing Libraries

In [413]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 2 Importing Data

In [416]:
path = r'C:\Users\elena\Documents\06.2025 climatewins machine learning'

In [418]:
# Upload original data

In [420]:
climate = pd.read_csv(os.path.join(path, '02 Data', 'Original data', 'Dataset-weather-prediction-dataset-processed.csv'), index_col = False)

In [422]:
climate.head()

DATE  MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0  19600101      1                  7               2.1            0.85   
1  19600102      1                  6               2.1            0.84   
2  19600103      1                  8               2.1            0.90   
3  19600104      1                  3               2.1            0.92   
4  19600105      1                  6               2.1            0.95   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0           1.018                    0.32                 0.09   
1           1.018                    0.36                 1.05   
2           1.018                    0.18                 0.30   
3           1.018                    0.58                 0.00   
4           1.018                    0.65                 0.14   

   BASEL_snow_depth  BASEL_sunshine  ...  VALENTIA_cloud_cover  \
0                 0             0.7  ...                     5   
1                 0             1.1  ...                     7   
2                 0             0.0  ...                     7   
3                 0             4.1  ...                     7   
4                 0             5.4  ...                     3   

   VALENTIA_humidity  VALENTIA_pressure  VALENTIA_global_radiation  \
0               0.88             1.0003                       0.45   
1               0.91             1.0007                       0.25   
2               0.91             1.0096                       0.17   
3               0.86             1.0184                       0.13   
4               0.80             1.0328                       0.46   

   VALENTIA_precipitation  VALENTIA_snow_depth  VALENTIA_sunshine  \
0                    0.34                    0                4.7   
1                    0.84                    0                0.7   
2                    0.08                    0                0.1   
3                    0.98                    0                0.0   
4                    0.00                    0                5.7   

   VALENTIA_temp_mean  VALENTIA_temp_min  VALENTIA_temp_max  
0                 8.5                6.0               10.9  
1                 8.9                5.6               12.1  
2                10.5                8.1               12.9  
3                 7.4                7.3               10.6  
4                 5.7                3.0                8.4  

[5 rows x 170 columns]

In [424]:
# Import the pleasant weather data.
pleasantweather = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

# 3 Data Wrangling

In [427]:
climate.columns

Index(['DATE', 'MONTH', 'BASEL_cloud_cover', 'BASEL_wind_speed',
       'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation',
       'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine',
       ...
       'VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure',
       'VALENTIA_global_radiation', 'VALENTIA_precipitation',
       'VALENTIA_snow_depth', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=170)

In [429]:
# Drop weather stations for which pleasant/unpleasant classification is not available

In [431]:
cols_to_drop = [col for col in climate.columns if any(city in col for city in ['GDANSK', 'ROMA', 'TOURS'])]


In [433]:
climate = climate.drop(columns=cols_to_drop)

In [435]:
climate.shape

(22950, 149)

In [437]:
# Extract the different observation types

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [439]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    # Select columns related to the current observation type
    columns = [col for col in climate.columns if col.endswith(obs)]
    
    # Count the number of stations (i.e., the number of columns) for the current observation type
    station_counts[obs] = len(columns)

# Print the count of stations for each observation type
print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")

Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


Wind speed and snow depth data is missing for a number of stations, so will remove these factors.

In [442]:
# Get a list of columns containing 'wind_speed' or 'snow_depth'
cols_to_drop = [col for col in climate.columns if '_wind_speed' in col or '_snow_depth' in col]

# Drop the columns
climate = climate.drop(cols_to_drop, axis=1)

In [444]:
climate.shape

(22950, 134)

Data from one location is missing for the following observation types: cloud cover, humidity and pressure.
Need to find out which stations are missing a column

In [447]:
stationParameters = {}
allParameters = set()

pairs = [p.split('_', 1) for p in climate.columns]

for pair in pairs:
    if len(pair) < 2: continue
    if pair[0] not in stationParameters:
        stationParameters[pair[0]] = set()
    stationParameters[pair[0]].add(pair[1])
    allParameters.add(pair[1])

stationParameters.keys()

dict_keys(['BASEL', 'BELGRADE', 'BUDAPEST', 'DEBILT', 'DUSSELDORF', 'HEATHROW', 'KASSEL', 'LJUBLJANA', 'MAASTRICHT', 'MADRID', 'MUNCHENB', 'OSLO', 'SONNBLICK', 'STOCKHOLM', 'VALENTIA'])

In [449]:
for station, parameters in stationParameters.items():
    if allParameters - parameters:
        print(station, "Missing:",  allParameters - parameters)

KASSEL Missing: {'cloud_cover'}
MUNCHENB Missing: {'pressure'}
STOCKHOLM Missing: {'humidity'}


In [451]:
# Create missing Kessel cloud cover and insert into the appropriate place.

In [453]:
# Step 1: Copy the column and assign to a new one
climate['KASSEL_cloud_cover'] = climate['LJUBLJANA_cloud_cover'].copy()

In [455]:
# Step 2: Get list of column names
cols = list(climate.columns)

# Step 3: Find index of the column *after which* to insert the new one
insert_after = 'HEATHROW_temp_max'
insert_at = cols.index(insert_after) + 1

# Step 4: Remove 'KASSEL_cloud_cover' from its current position (end of list)
cols.remove('KASSEL_cloud_cover')

# Step 5: Insert it at the desired position
cols.insert(insert_at, 'KASSEL_cloud_cover')

# Step 6: Reorder the DataFrame
climate = climate[cols]

In [457]:
# Create missing 'MUNCHENB_pressure' and insert after 'MUNCHENB_humidity'

In [459]:
# Step 1: Copy the column and assign to a new one
climate['MUNCHENB_pressure'] = climate['SONNBLICK_pressure'].copy()

In [461]:
# Step 2: Get list of column names
cols = list(climate.columns)

# Step 3: Find index of the column *after which* to insert the new one
insert_after = 'MUNCHENB_humidity'
insert_at = cols.index(insert_after) + 1

# Step 4: Remove 'MUNCHENB_pressure' from its current position (end of list)
cols.remove('MUNCHENB_pressure')

# Step 5: Insert it at the desired position
cols.insert(insert_at, 'MUNCHENB_pressure')

# Step 6: Reorder the DataFrame
climate = climate[cols]

In [463]:
# Create missing 'STOCKHOLM_humidity' and insert after STOCKHOLM_cloud_cover'

In [465]:
# Step 1: Copy the column and assign to a new one
climate['STOCKHOLM_humidity'] = climate['OSLO_humidity'].copy()

In [467]:
# Step 2: Get list of column names
cols = list(climate.columns)

# Step 3: Find index of the column *after which* to insert the new one
insert_after = 'STOCKHOLM_cloud_cover'
insert_at = cols.index(insert_after) + 1

# Step 4: Remove 'MUNCHENB_pressure' from its current position (end of list)
cols.remove('STOCKHOLM_humidity')

# Step 5: Insert it at the desired position
cols.insert(insert_at, 'STOCKHOLM_humidity')

# Step 6: Reorder the DataFrame
climate = climate[cols]

In [469]:
list(climate.columns)

['DATE',
 'MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSS

In [471]:
climate.shape

(22950, 137)

In [473]:
pleasantweather.shape

(22950, 16)

In [475]:
# Drop unnecessary columns
climate.drop(['DATE', 'MONTH'], axis=1, inplace=True)

In [477]:
pleasantweather.drop(['DATE'], axis=1, inplace=True)

In [481]:
# Export cleaned dataset
climate.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_cleaned1.csv'), index=False)

# 4 Reshape Data for Deep Learning

In [484]:
# Creat an 'X' matrix by reloading and naming our data 'X'
X=pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'weather_cleaned1.csv'), index_col=False)

In [486]:
X.head()

BASEL_cloud_cover  BASEL_humidity  BASEL_pressure  BASEL_global_radiation  \
0                  7            0.85           1.018                    0.32   
1                  6            0.84           1.018                    0.36   
2                  8            0.90           1.018                    0.18   
3                  3            0.92           1.018                    0.58   
4                  6            0.95           1.018                    0.65   

   BASEL_precipitation  BASEL_sunshine  BASEL_temp_mean  BASEL_temp_min  \
0                 0.09             0.7              6.5             0.8   
1                 1.05             1.1              6.1             3.3   
2                 0.30             0.0              8.5             5.1   
3                 0.00             4.1              6.3             3.8   
4                 0.14             5.4              3.0            -0.7   

   BASEL_temp_max  BELGRADE_cloud_cover  ...  STOCKHOLM_temp_max  \
0            10.9                     1  ...                 4.9   
1            10.1                     6  ...                 5.0   
2             9.9                     6  ...                 4.1   
3            10.6                     8  ...                 2.3   
4             6.0                     8  ...                 4.3   

   VALENTIA_cloud_cover  VALENTIA_humidity  VALENTIA_pressure  \
0                     5               0.88             1.0003   
1                     7               0.91             1.0007   
2                     7               0.91             1.0096   
3                     7               0.86             1.0184   
4                     3               0.80             1.0328   

   VALENTIA_global_radiation  VALENTIA_precipitation  VALENTIA_sunshine  \
0                       0.45                    0.34                4.7   
1                       0.25                    0.84                0.7   
2                       0.17                    0.08                0.1   
3                       0.13                    0.98                0.0   
4                       0.46                    0.00                5.7   

   VALENTIA_temp_mean  VALENTIA_temp_min  VALENTIA_temp_max  
0                 8.5                6.0               10.9  
1                 8.9                5.6               12.1  
2                10.5                8.1               12.9  
3                 7.4                7.3               10.6  
4                 5.7                3.0                8.4  

[5 rows x 135 columns]

In [488]:
y = pleasantweather

In [490]:
X.shape

(22950, 135)

In [492]:
# Turn X and y into arrays
X = np.array(X)
y = np.array(y)

In [494]:
X = X.reshape(-1,15,9)

In [496]:
# Verify Shape
X.shape

(22950, 15, 9)

In [498]:
# Verify Shape
y.shape

(22950, 15)

In [500]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [502]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


# 5 Keras Model CNN

## 5.1 Model 1

In [505]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [507]:
model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_15 (Conv1D)              │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_13 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

In [561]:
# Compile and run Model 1

In [510]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [512]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
1076/1076 - 2s - 2ms/step - accuracy: 0.1520 - loss: 0.2547
Epoch 2/30
1076/1076 - 2s - 1ms/step - accuracy: 0.1673 - loss: 0.1869
Epoch 3/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1721 - loss: 0.1668
Epoch 4/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1832 - loss: 0.1528
Epoch 5/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1825 - loss: 0.1404
Epoch 6/30
1076/1076 - 1s - 1ms/step - accuracy: 0.1934 - loss: 0.1297
Epoch 7/30
1076/1076 - 2s - 1ms/step - accuracy: 0.2128 - loss: 0.1204
Epoch 8/30
1076/1076 - 2s - 1ms/step - accuracy: 0.2405 - loss: 0.1121
Epoch 9/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2351 - loss: 0.1039
Epoch 10/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2631 - loss: 0.0973
Epoch 11/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2526 - loss: 0.0904
Epoch 12/30
1076/1076 - 2s - 1ms/step - accuracy: 0.2656 - loss: 0.0837
Epoch 13/30
1076/1076 - 2s - 1ms/step - accuracy: 0.2750 - loss: 0.0776
Epoch 14/30
1076/1076 - 1s - 1ms/step - accuracy: 0.2886 - loss: 0.0717
E

### 5.1.1 Model 1 Confusion Matrix

In [515]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [517]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [519]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         938       412        51      46          56        65      26   
BELGRADE        0       577        22       3          10        11       4   
BUDAPEST        0         1        33       3           2         7       2   
DEBILT          0         0         1      16           1         3       0   
DUSSELDORF      0         0         0       0           2         3       0   
HEATHROW        0         0         0       0           0        12       0   
KASSEL          0         0         0       0           0         0       3   
LJUBLJANA       0         0         0       0           0         0       0   
MAASTRICHT      0         0         0       0           0         0       0   
MADRID          0         0         0       0           0         0       0   
MUNCHENB 

In [521]:
from sklearn.metrics import f1_score

# Step 1: Predict probabilities
y_pred_prob = model.predict(X_test)

# Step 2: Convert probabilities to binary predictions (threshold at 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# Step 3: Compute weighted F1 score
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
Weighted F1 Score: 0.9637


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [523]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1400
           1       0.92      1.00      0.96      1962
           2       0.96      0.99      0.98      1838
           3       0.91      0.99      0.95      1101
           4       0.95      0.99      0.97      1231
           5       0.92      0.99      0.95      1168
           6       0.93      0.98      0.96       923
           7       0.94      1.00      0.97      1543
           8       0.91      0.99      0.95      1176
           9       0.96      1.00      0.98      2570
          10       0.96      1.00      0.98      1192
          11       0.91      1.00      0.95       859
          12       0.00      0.00      0.00         0
          13       0.97      0.99      0.98       972
          14       0.81      0.91      0.86       276

   micro avg       0.94      0.99      0.96     18211
   macro avg       0.87      0.92      0.89     18211
weighted avg       0.94   

## 5.2 Model 2

In [527]:
# Try bigger batch size 

In [529]:
epochs = 30
batch_size = 32
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [531]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [533]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
538/538 - 2s - 3ms/step - accuracy: 0.1384 - loss: 0.3533
Epoch 2/30
538/538 - 1s - 1ms/step - accuracy: 0.1592 - loss: 0.2162
Epoch 3/30
538/538 - 1s - 1ms/step - accuracy: 0.1619 - loss: 0.1957
Epoch 4/30
538/538 - 1s - 1ms/step - accuracy: 0.1648 - loss: 0.1818
Epoch 5/30
538/538 - 1s - 1ms/step - accuracy: 0.1696 - loss: 0.1710
Epoch 6/30
538/538 - 1s - 1ms/step - accuracy: 0.1731 - loss: 0.1617
Epoch 7/30
538/538 - 1s - 1ms/step - accuracy: 0.1753 - loss: 0.1541
Epoch 8/30
538/538 - 1s - 1ms/step - accuracy: 0.1775 - loss: 0.1455
Epoch 9/30
538/538 - 1s - 1ms/step - accuracy: 0.1785 - loss: 0.1391
Epoch 10/30
538/538 - 1s - 1ms/step - accuracy: 0.1811 - loss: 0.1329
Epoch 11/30
538/538 - 1s - 1ms/step - accuracy: 0.1855 - loss: 0.1260
Epoch 12/30
538/538 - 1s - 1ms/step - accuracy: 0.1832 - loss: 0.1205
Epoch 13/30
538/538 - 1s - 1ms/step - accuracy: 0.1860 - loss: 0.1152
Epoch 14/30
538/538 - 1s - 1ms/step - accuracy: 0.1904 - loss: 0.1098
Epoch 15/30
538/538 - 1s - 1m

In [535]:
# Step 1: Predict probabilities
y_pred_prob = model.predict(X_test)

# Step 2: Convert probabilities to binary predictions (threshold at 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# Step 3: Compute weighted F1 score
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step
Weighted F1 Score: 0.9545


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


F1 score is worse, so return to previous batch size and try to increase epochs. 

## 5.3 Model 3

In [539]:
epochs = 60
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [541]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_17 (Conv1D)              │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 14, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,831 (11.06 KB)

 Trainable params: 2,831 (11.06 KB)

 Non-trainable params: 0 (0.00 B)

In [543]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [545]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1379 - loss: 0.2743
Epoch 2/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1588 - loss: 0.1976
Epoch 3/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1706 - loss: 0.1763
Epoch 4/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1935 - loss: 0.1622
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1989 - loss: 0.1497
Epoch 6/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2025 - loss: 0.1393
Epoch 7/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1992 - loss: 0.1292
Epoch 8/60
1076/1076 - 1s - 1ms/step - accuracy: 0.2027 - loss: 0.1208
Epoch 9/60
1076/1076 - 1s - 1ms/step - accuracy: 0.2058 - loss: 0.1122
Epoch 10/60
1076/1076 - 1s - 1ms/step - accuracy: 0.1967 - loss: 0.1044
Epoch 11/60
1076/1076 - 1s - 1ms/step - accuracy: 0.2068 - loss: 0.0959
Epoch 12/60
1076/1076 - 1s - 1ms/step - accuracy: 0.2076 - loss: 0.0886
Epoch 13/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2173 - loss: 0.0819
Epoch 14/60
1076/1076 - 1s - 1ms/step - accuracy: 0.2142 - loss: 0.0753
E

In [547]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step
Weighted F1 Score: 0.9848


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Improved model performance! Keep this change.

## 5.4 Model 4

Now let's try changing model architecture. First add more filters. More filters = more feature power

In [552]:
epochs = 60
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [554]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_18 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_18 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [556]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [558]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1598 - loss: 0.2650
Epoch 2/60
1076/1076 - 2s - 1ms/step - accuracy: 0.1887 - loss: 0.1895
Epoch 3/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1885 - loss: 0.1690
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1942 - loss: 0.1543
Epoch 5/60
1076/1076 - 2s - 1ms/step - accuracy: 0.1997 - loss: 0.1422
Epoch 6/60
1076/1076 - 2s - 1ms/step - accuracy: 0.1983 - loss: 0.1308
Epoch 7/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2136 - loss: 0.1209
Epoch 8/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2347 - loss: 0.1118
Epoch 9/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2421 - loss: 0.1030
Epoch 10/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2403 - loss: 0.0954
Epoch 11/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2497 - loss: 0.0880
Epoch 12/60
1076/1076 - 2s - 1ms/step - accuracy: 0.2553 - loss: 0.0816
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2519 - loss: 0.0762
Epoch 14/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2449 - loss: 0.0704
E

In [564]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9862


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Obtained another small improvement in model performance

## 5.5 Model 5

Let's increase kernel size to 5

In [572]:
epochs = 60
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=5, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [574]:
model.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_19 (Conv1D)              │ (None, 11, 64)         │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 11, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_19 (MaxPooling1D) │ (None, 5, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_17 (Flatten)            │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 15)             │         1,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,199 (20.31 KB)

 Trainable params: 5,199 (20.31 KB)

 Non-trainable params: 0 (0.00 B)

In [576]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [578]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 3s - 3ms/step - accuracy: 0.1716 - loss: 0.2563
Epoch 2/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1857 - loss: 0.1999
Epoch 3/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1879 - loss: 0.1832
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1852 - loss: 0.1717
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1863 - loss: 0.1619
Epoch 6/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1950 - loss: 0.1545
Epoch 7/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1990 - loss: 0.1467
Epoch 8/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1980 - loss: 0.1394
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2011 - loss: 0.1329
Epoch 10/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1986 - loss: 0.1276
Epoch 11/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2015 - loss: 0.1215
Epoch 12/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2060 - loss: 0.1159
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1978 - loss: 0.1124
Epoch 14/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2059 - loss: 0.1065
E

In [580]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9580


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model performed worse with bigger kernel size, so original kernel size is better for the current problem.

## 5.6 Model 6: Extra Convolution Layer

In [584]:
epochs = 60
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()

# 1st Convolutional layer
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))

# 2nd Convolutional layer
model.add(Conv1D(32, kernel_size=2, activation='relu'))

model.add(Dense(16, activation='relu'))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [586]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_20 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 13, 32)         │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 13, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 6, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_18 (Flatten)            │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 15)             │         1,455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,327 (28.62 KB)

 Trainable params: 7,327 (28.62 KB)

 Non-trainable params: 0 (0.00 B)

In [588]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [590]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 4s - 3ms/step - accuracy: 0.1599 - loss: 0.2623
Epoch 2/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1997 - loss: 0.1882
Epoch 3/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2554 - loss: 0.1639
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2402 - loss: 0.1438
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2146 - loss: 0.1267
Epoch 6/60
1076/1076 - 3s - 2ms/step - accuracy: 0.2233 - loss: 0.1131
Epoch 7/60
1076/1076 - 3s - 3ms/step - accuracy: 0.2092 - loss: 0.1007
Epoch 8/60
1076/1076 - 3s - 3ms/step - accuracy: 0.1971 - loss: 0.0911
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1957 - loss: 0.0832
Epoch 10/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1898 - loss: 0.0746
Epoch 11/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1935 - loss: 0.0683
Epoch 12/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1898 - loss: 0.0623
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1936 - loss: 0.0579
Epoch 14/60
1076/1076 - 3s - 3ms/step - accuracy: 0.1988 - loss: 0.0535
E

In [592]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9750


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Adding the Convolution layer did not improve the performance. Weighted F1 score is worse than for several previous models.
Next I will try using Dropout after the convolution layers

## 5.7 Model 7: Extra Convolution Layer and Dropout

In [596]:
epochs = 60
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()

# 1st Convolutional layer
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))

# 2nd Convolutional layer
model.add(Conv1D(32, kernel_size=2, activation='relu'))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [598]:
model.summary()

Model: "sequential_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_22 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_23 (Conv1D)              │ (None, 13, 32)         │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 13, 16)         │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 16)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 6, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 15)             │         1,455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,327 (28.62 KB)

 Trainable params: 7,327 (28.62 KB)

 Non-trainable params: 0 (0.00 B)

In [600]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [602]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 4s - 3ms/step - accuracy: 0.1508 - loss: 0.2731
Epoch 2/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2208 - loss: 0.2088
Epoch 3/60
1076/1076 - 3s - 2ms/step - accuracy: 0.2528 - loss: 0.1836
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2391 - loss: 0.1666
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2512 - loss: 0.1520
Epoch 6/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2465 - loss: 0.1389
Epoch 7/60
1076/1076 - 3s - 3ms/step - accuracy: 0.2466 - loss: 0.1274
Epoch 8/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2407 - loss: 0.1171
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2307 - loss: 0.1100
Epoch 10/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2401 - loss: 0.1023
Epoch 11/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2548 - loss: 0.0956
Epoch 12/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2588 - loss: 0.0901
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2720 - loss: 0.0862
Epoch 14/60
1076/1076 - 3s - 3ms/step - accuracy: 0.3129 - loss: 0.0820
E

In [604]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9563


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Using Dropout did not improve weighted F1. 

## 5.8 Model 8: Extra Pooling Layer

To try to find the model with best performance, I will try to add an extra Pooling layer incread of the extra Convolution layer

In [609]:
epochs = 60
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()

# Convolutional layer
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))

model.add(Dense(16, activation='relu'))

model.add(MaxPooling1D())
# Extra layer
model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [611]:
model.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_24 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_22 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_23 (MaxPooling1D) │ (None, 3, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_20 (Flatten)            │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 15)             │           735 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,991 (11.68 KB)

 Trainable params: 2,991 (11.68 KB)

 Non-trainable params: 0 (0.00 B)

In [613]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [615]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 3s - 2ms/step - accuracy: 0.1073 - loss: 0.3042
Epoch 2/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1497 - loss: 0.2483
Epoch 3/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1620 - loss: 0.2340
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1681 - loss: 0.2211
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1733 - loss: 0.2120
Epoch 6/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1765 - loss: 0.2040
Epoch 7/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1833 - loss: 0.1960
Epoch 8/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1842 - loss: 0.1883
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1849 - loss: 0.1826
Epoch 10/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1855 - loss: 0.1776
Epoch 11/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1857 - loss: 0.1727
Epoch 12/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1882 - loss: 0.1687
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1896 - loss: 0.1634
Epoch 14/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1903 - loss: 0.1597
E

In [617]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9106


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Adding an extra pooling layer to the model produced a considerably lower weighted F1, so this change is not beneficial. Let's go back to the previous model version. 

## 5.9 Model 9: More Filters

Previously, adding more filters improved model performance. Let's add even more and see if there will be more improvement.

In [622]:
epochs = 60
batch_size = 16
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [624]:
model.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_25 (Conv1D)              │ (None, 14, 128)        │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 14, 16)         │         2,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_24 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,191 (24.18 KB)

 Trainable params: 6,191 (24.18 KB)

 Non-trainable params: 0 (0.00 B)

In [626]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [628]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/60
1076/1076 - 3s - 3ms/step - accuracy: 0.1412 - loss: 0.2524
Epoch 2/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1703 - loss: 0.1845
Epoch 3/60
1076/1076 - 2s - 2ms/step - accuracy: 0.1893 - loss: 0.1616
Epoch 4/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2426 - loss: 0.1421
Epoch 5/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2590 - loss: 0.1271
Epoch 6/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2695 - loss: 0.1148
Epoch 7/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2834 - loss: 0.1042
Epoch 8/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2823 - loss: 0.0929
Epoch 9/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2887 - loss: 0.0861
Epoch 10/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2850 - loss: 0.0773
Epoch 11/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2857 - loss: 0.0699
Epoch 12/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2741 - loss: 0.0633
Epoch 13/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2590 - loss: 0.0585
Epoch 14/60
1076/1076 - 2s - 2ms/step - accuracy: 0.2548 - loss: 0.0539
E

In [630]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9791


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Weighted F1 score is not the best, so do not accept the change.

## 5.10 Model 10: More Epochs

Previously, increasing epochs improved the model, so will try to increase again.

In [666]:
epochs = 90
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='sigmoid')) 

C:\Users\elena\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [668]:
model.summary()

Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_29 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 14, 16)         │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_28 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_25 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,951 (15.43 KB)

 Trainable params: 3,951 (15.43 KB)

 Non-trainable params: 0 (0.00 B)

In [670]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [672]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/90
1076/1076 - 2s - 2ms/step - accuracy: 0.1339 - loss: 0.2615
Epoch 2/90
1076/1076 - 2s - 2ms/step - accuracy: 0.1868 - loss: 0.1977
Epoch 3/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2016 - loss: 0.1737
Epoch 4/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2013 - loss: 0.1576
Epoch 5/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2065 - loss: 0.1419
Epoch 6/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2124 - loss: 0.1306
Epoch 7/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2243 - loss: 0.1200
Epoch 8/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2221 - loss: 0.1120
Epoch 9/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2187 - loss: 0.1031
Epoch 10/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2257 - loss: 0.0963
Epoch 11/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2387 - loss: 0.0890
Epoch 12/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2375 - loss: 0.0829
Epoch 13/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2417 - loss: 0.0779
Epoch 14/90
1076/1076 - 2s - 2ms/step - accuracy: 0.2400 - loss: 0.0731
E

In [674]:
# Calculating F1 to evaluate model performance

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Weighted F1 Score: 0.9874


C:\Users\elena\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


This step did not produce a better model. We will stop here and report results for the best model achieved so far.   

In [677]:
# Confusion Matrix

In [2]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [681]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         437       163        61      79          89       106      65   
BELGRADE        0       406       113       7          27        19      21   
BUDAPEST        0         0        66       4           2         7       3   
DEBILT          0         0         0      21           4         3       1   
DUSSELDORF      0         0         0       0           3         2       0   
HEATHROW        0         0         0       0           0        14       0   
KASSEL          0         0         0       0           0         0       3   
LJUBLJANA       0         0         0       0           0         0       0   
MAASTRICHT      0         0         0       0           0         0       0   
MADRID          0         0         0       0           0         0       0   
MUNCHENB 